* Useful information about pdb file format

https://www.cgl.ucsf.edu/chimera/docs/UsersGuide/tutorials/pdbintro.html

https://www.wwpdb.org/documentation/file-format-content/format33/sect3.html

In [7]:
from Bio.PDB import *
from Bio import SeqIO
import numpy as np
import numpy.matlib
from sklearn import metrics

#filename = '3ne4'
filename = '3t1p'

### Method to find the dbref chain length (this is database info), differences in SEQADV 
* Pay attention to the reference start and end in comparison to the pdb start end residue positions
* Update function 1/6/20, need start and end position for updating matrix and pdb chain length

In [8]:
#chain_lengths (list of lists) -> start pos, end pos, chain length
pdb_chains = []
ref_chains = []

with open(f'pdb_files/{filename}.pdb') as handle:
    #reads every line in the file
    for line in handle:
        #Locate dbref line
        rec_name = line[0:6].strip() 
        if rec_name == "DBREF":
            pdb_start = int(line[14:18])
            pdb_end = int(line[20:24])
            pdb_chains.append([pdb_start,pdb_end,pdb_end-pdb_start]) #end position does not have an atom measurement
            
            ref_start = int(line[55:60])
            ref_end = int(line[62:67])
            ref_chains.append([ref_start, ref_end, ref_end])

print(pdb_chains)
print(ref_chains)

[[24, 394, 370]]
[[48, 418, 418]]


### Using reference chain length, fetch ca position for each residue

In [9]:
parser = PDBParser()
structure = parser.get_structure('cftr',f'pdb_files/{filename}.pdb')
chains=[]

positions = {} #positions of ca atom for every residue in chain, if not in pdb file then set to NaN
for model in structure:
    for chain, ref_length, pdb_length in zip(model,ref_chains,pdb_chains):
        chains.append(chain)
        positions[f'chain {chain.id}'] = np.empty([ref_length[2],3])
        positions[f'chain {chain.id}'][:] = np.nan
        for res in chain.get_residues():
            pos = res.get_full_id()[3][1]
            
            if pos>pdb_length[1]:
                 break          
#             if pos>positions[f'chain {chain.id}'].shape[0]:
#                 break
            else:
                adjusted_pos = pos-1+ref_length[0]-pdb_length[0] #the pos-1 is because we are indexed 0 and data is indexed 1
                positions[f'chain {chain.id}'][adjusted_pos,:] = res['CA'].coord 

### use sklearn pairwise distances to compute every pairwise distance

In [4]:
pairwise_distances = {}
for chain in positions:
    pairwise_distances[chain] = [] #list for full matrix and column form
    pairwise = metrics.pairwise_distances(positions[chain],positions[chain],metric='nan_euclidean',force_all_finite='allow-nan') #compute all pairwise distances
    
    #append pairwise matrix
    pairwise_distances[chain].append(pairwise)
    
    #column form
#     indxs = np.matlib.repmat(range(1,pairwise.shape[0]+1),pairwise.shape[0],1).T
#     indxs =  np.reshape(indxs,pairwise.size)[:,None]
#     vals = np.reshape(pairwise,pairwise.size)[:,None]
#     pairwise_distances[chain].append(np.hstack((vals,indxs)))

In [5]:
pairwise_distances[chain][0].shape

(418, 418)

In [6]:
chain = 'chain A'
np.savetxt(f'distances/{filename}_dist_mat.txt',pairwise_distances[chain][0],fmt='%.5f',delimiter='\t')
# np.savetxt(f'distances/{filename}_dist_cols.txt',pairwise_distances[chain][1],fmt='%.5f')